In [2]:
# This code implements a hierarchical MNL choice model
# with a multivariate normal distribution of heterogeneity via Stan.

# Preamble ----------------------------------------------------------------
# Load libraries.
library(loo)
library(rstan)
library(tidyverse)
library(mvtnorm)
library(rstan)
library(bayesplot)

# Set Stan options.
rstan_options(auto_write = TRUE)
options(mc.cores = parallel::detectCores())

# Parameter Recovery ------------------------------------------------------
nresp <- 100    # Number of respondents.
nscns <- 10     # Number of choice scenarios for each respondent.
nalts <- 4      # Number of alternatives in each choice scenario.
nlvls <- 12     # Number of attribute levels for each alternative.
ncovs <- 1      # Number of covariates for each respondent.

# True Gamma (ncovs x nlvls) values and Vbeta (nlvls x nlvls) values.
Gamma <- matrix(runif(ncovs * nlvls, -3, 4), nrow = nlvls, ncol = ncovs)
Vbeta <- diag(nlvls) + .5 * matrix(1, nrow = nlvls, ncol = 1) %*% t(matrix(1, nrow = nlvls, ncol = 1))

# Generate data.
Y <- matrix(nrow = nresp, ncol = nscns)
X <- array(dim = c(nresp, nscns, nalts, nlvls))
Z <- matrix(nrow = ncovs, ncol = nresp)
Beta <- matrix(nrow = nlvls, ncol = nresp)
for (resp in 1:nresp) {
  # Generate covariates for the distribution of heterogeneity.
  z_resp <- 1
  if (ncovs > 1) z_resp <- c(z_resp, round(runif(ncovs - 1)))
  
  # Generate individual-level betas.
  beta <- rmvnorm(1, mean = Gamma %*% z_resp, sigma = Vbeta)
    
  # Compute the latent utility a scenario at a time.
  for (scn in 1:nscns) {
    # Generate the design matrix for the given scenario.
    X_scn <- matrix(round(runif(nalts * nlvls)), nrow = nalts, ncol = nlvls)
    
    # Compute and the latent utility for each alternative and find the max.
    U_scn <- X_scn %*% as.vector(beta) + matrix((-log(-log(runif(nalts)))), ncol = 1)
    
    # Save each scenario's data.
    Y[resp, scn] <- which(U_scn == max(U_scn))
    X[resp, scn, , ] <- X_scn
  }
  
  # Save out each respondent's data.
  Z[, resp] <- as.vector(z_resp)
  Beta[, resp] <- as.vector(beta)
}

# Save data in a list.
Data <- list(nresp = nresp, nscns = nscns, nalts = nalts, nlvls = nlvls, ncovs = ncovs,
             Y = Y, X = X, Z = Z, Gamma = Gamma, Vbeta = Vbeta, Beta = Beta)

log_lik_list <- list()
for (k in 1:K){
    # Fit the k-th model with Stan
    fit <- stan("./MODELS/HBMNL_1.1.stan", data=list(y=y, X=X[,k], n=length(y), p=1))
    log_lik_list[[k]] <- extract(fit)[["log_lik"]]
}

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  1.4.2     ✔ purrr   0.2.5
✔ tidyr   0.8.1     ✔ dplyr   0.7.6
✔ readr   1.1.1     ✔ stringr 1.3.1
✔ tibble  1.4.2     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks rstan::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()


ERROR: Error: package or namespace load failed for ‘mvtnorm’ in dyn.load(file, DLLpath = DLLpath, ...):
 unable to load shared object '/Users/derekmiller/anaconda3/lib/R/library/mvtnorm/libs/mvtnorm.so':
  dlopen(/Users/derekmiller/anaconda3/lib/R/library/mvtnorm/libs/mvtnorm.so, 6): Library not loaded: @rpath/libintl.9.dylib
  Referenced from: /Users/derekmiller/anaconda3/lib/R/library/mvtnorm/libs/mvtnorm.so
  Reason: image not found
